In [1]:
!pip install gdown

# Importing Libraries

In [2]:
import pandas as pd
import re
import gdown

# Importing Data

In [3]:
gdown.download("https://drive.google.com/uc?id=1Gpz4iB6xcrqcvjBBn6FsGE92iP1P0Ljn", '/kaggle/working/com.xlsx', quiet=False)
print('/kaggle/working/com.xlsx')

Downloading...
From: https://drive.google.com/uc?id=1Gpz4iB6xcrqcvjBBn6FsGE92iP1P0Ljn
To: /kaggle/working/com.xlsx
100%|██████████| 389k/389k [00:00<00:00, 49.2MB/s]

/kaggle/working/com.xlsx


In [4]:
gdown.download("https://drive.google.com/uc?id=1-PWmMD8b7EG7SQM3WxIrVbrb2g9eDz5E", '/kaggle/working/non.csv', quiet=False)
print('/kaggle/working/non.csv')

Downloading...
From: https://drive.google.com/uc?id=1-PWmMD8b7EG7SQM3WxIrVbrb2g9eDz5E
To: /kaggle/working/non.csv
100%|██████████| 435k/435k [00:00<00:00, 90.1MB/s]

/kaggle/working/non.csv


# Data Preprocessing

## com

In [5]:
com = pd.read_excel("/kaggle/working/com.xlsx")
com.head()

,Date,Tweet_id,Text,Compartive,EN1,EN2,Sentiment,PreferedEntity,idx
0,2022-05-08 20:35:40+00:00,1523400000000000000,@Abdulaziz_Hmadi لابتوب اسوس افضل من لابتوبات...,com,اسوس,ابل,n,ابل,0
1,2022-04-17 19:33:58+00:00,1515780000000000000,@tareqtareq1951 @DalelkoinRiyadh اسوس افضل من ...,com,اسوس,NaN,p,اسوس,1
2,2022-04-04 01:55:43+00:00,1510800000000000000,@zoro00710 كلهم نفس المواصفات مافي فرق\nإسوس ا...,com,إسوس,HP,p,إسوس,2
3,2022-04-03 18:06:31+00:00,1510680000000000000,@b4raa20 التخصصات الهندسية بالذات الحاسب والبر...,com,اسوس,NaN,p,اسوس,3
4,2022-03-12 14:10:56+00:00,1502650000000000000,@wilchert اسوس افضل من ناحية التنوع والأسعار و...,com,اسوس,NaN,p,اسوس,4


In [6]:
com = com.drop(['Date', 'Tweet_id', 'idx', 'Sentiment', 'PreferedEntity'], axis=1)
com.head()

,Text,Compartive,EN1,EN2
0,@Abdulaziz_Hmadi لابتوب اسوس افضل من لابتوبات...,com,اسوس,ابل
1,@tareqtareq1951 @DalelkoinRiyadh اسوس افضل من ...,com,اسوس,NaN
2,@zoro00710 كلهم نفس المواصفات مافي فرق\nإسوس ا...,com,إسوس,HP
3,@b4raa20 التخصصات الهندسية بالذات الحاسب والبر...,com,اسوس,NaN
4,@wilchert اسوس افضل من ناحية التنوع والأسعار و...,com,اسوس,NaN


In [7]:
com = com.rename(columns={"Text": "text", "Compartive": "comp", "EN1": "en1", "EN2": "en2",})
com.head()

,text,comp,en1,en2
0,@Abdulaziz_Hmadi لابتوب اسوس افضل من لابتوبات...,com,اسوس,ابل
1,@tareqtareq1951 @DalelkoinRiyadh اسوس افضل من ...,com,اسوس,NaN
2,@zoro00710 كلهم نفس المواصفات مافي فرق\nإسوس ا...,com,إسوس,HP
3,@b4raa20 التخصصات الهندسية بالذات الحاسب والبر...,com,اسوس,NaN
4,@wilchert اسوس افضل من ناحية التنوع والأسعار و...,com,اسوس,NaN


In [8]:
# Duplicate Cleaning
com.duplicated().sum()

6

In [9]:
com = com.drop_duplicates()
com.duplicated().sum()

0

In [10]:
# Nulls Cleaning
com.isnull().sum()

text       1
comp       2
en1      491
en2     1009
dtype: int64

In [11]:
com[com[['text', 'comp']].isnull().any(axis=1)]

,text,comp,en1,en2
1511,NaN,com,"نوت , هواوي",NaN
1803,#محمد_عبده_افضل_من_طلال_مداح\n#ايفون_افضل_من_ج...,NaN,NaN,NaN
2369,"""طلعت زكريا"" لـ دوت مصر: التكنولوجيا أضرت بالم...",NaN,NaN,NaN


In [12]:
com = com.dropna(subset=['text', 'comp'])
com.isnull().sum()

text       0
comp       0
en1      489
en2     1006
dtype: int64

In [13]:
# Comp Cleaning
com.comp.value_counts()

comp
com     2326
non      495
non        6
 non       1
com        1
Name: count, dtype: int64

In [14]:
com['comp'] = com['comp'].str.strip()
com.comp.value_counts()

comp
com    2327
non     502
Name: count, dtype: int64

In [15]:
com = com[com['comp'] == 'com']
com.comp.value_counts()

comp
com    2327
Name: count, dtype: int64

## non

In [16]:
non = pd.read_csv("/kaggle/working/non.csv")
non.head()

,Tweet Date,USER Location,Tweet Text,Compartive
0,2021-08-05 6:55:39,المريخ,@2BamBaa كارير ولا توشيبا 🙂,non
1,2021-08-04 23:16:08,NaN,@DaliaMo79101394 انا توشيبا على فكرة..😅😅,non
2,2021-08-04 21:38:40,"الشارقة, الامارات العربية المتحدة",هو انا لوحدي اللي مبحبش محمد هنيدي حتى منتجات ...,non
3,2021-08-04 21:09:45,NaN,@AboFlah_1 ولله انت قدوتي \nوالله محتاج هدا 20...,non
4,2021-08-04 16:54:22,NaN,@AlManeaCo هل متوفر في فروع الرياض غساله ملابس...,non


In [17]:
non = non.drop(['USER Location', 'Tweet Date'], axis=1)
non.head()

,Tweet Text,Compartive
0,@2BamBaa كارير ولا توشيبا 🙂,non
1,@DaliaMo79101394 انا توشيبا على فكرة..😅😅,non
2,هو انا لوحدي اللي مبحبش محمد هنيدي حتى منتجات ...,non
3,@AboFlah_1 ولله انت قدوتي \nوالله محتاج هدا 20...,non
4,@AlManeaCo هل متوفر في فروع الرياض غساله ملابس...,non


In [18]:
# columns modifying
non = non.rename(columns={"Tweet Text": "text", "Compartive": "comp"})
non["en1"] = None
non["en2"] = None
non.head()

,text,comp,en1,en2
0,@2BamBaa كارير ولا توشيبا 🙂,non,None,None
1,@DaliaMo79101394 انا توشيبا على فكرة..😅😅,non,None,None
2,هو انا لوحدي اللي مبحبش محمد هنيدي حتى منتجات ...,non,None,None
3,@AboFlah_1 ولله انت قدوتي \nوالله محتاج هدا 20...,non,None,None
4,@AlManeaCo هل متوفر في فروع الرياض غساله ملابس...,non,None,None


In [19]:
# duplicate cleaning
non.duplicated().sum()

6

In [20]:
non = non.drop_duplicates()
non.duplicated().sum()

0

In [21]:
# nulls cleaning
non.isnull().sum()

text       1
comp       0
en1     2273
en2     2273
dtype: int64

In [22]:
non = non.dropna(subset=['text'])
non.isnull().sum()

text       0
comp       0
en1     2272
en2     2272
dtype: int64

# Data Integration

In [23]:
df = pd.concat([com, non], ignore_index=True)
df.head()

,text,comp,en1,en2
0,@Abdulaziz_Hmadi لابتوب اسوس افضل من لابتوبات...,com,اسوس,ابل
1,@tareqtareq1951 @DalelkoinRiyadh اسوس افضل من ...,com,اسوس,NaN
2,@zoro00710 كلهم نفس المواصفات مافي فرق\nإسوس ا...,com,إسوس,HP
3,@b4raa20 التخصصات الهندسية بالذات الحاسب والبر...,com,اسوس,NaN
4,@wilchert اسوس افضل من ناحية التنوع والأسعار و...,com,اسوس,NaN


In [24]:
df.comp.value_counts()

comp
com    2327
non    2272
Name: count, dtype: int64

# Data Cleaning

In [25]:
# target encoding
df.comp = df.comp.map({'non': 0, 'com': 1})
df.comp.value_counts()

comp
1    2327
0    2272
Name: count, dtype: int64

In [26]:
df.shape

(4599, 4)

In [27]:
# preprocessing functions
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        "\U0001F1E0-\U0001F1FF"
        "\U0001F300-\U0001F5FF"
        "\U0001F600-\U0001F64F"
        "\U0001F680-\U0001F6FF"
        "\U0001F700-\U0001F77F"
        "\U0001F780-\U0001F7FF"
        "\U0001F800-\U0001F8FF"
        "\U0001F900-\U0001F9FF"
        "\U0001FA00-\U0001FA6F"
        "\U0001FA70-\U0001FAFF"
        "\U00002500-\U00002BEF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001F926-\U0001F937"
        "\U00010000-\U0010FFFF"
        "\u2640-\u2642" 
        "\u2600-\u2B55"
        "\u23e9-\u23ef"
        "\u231a-\u231b"
        "\u200d" 
        "\u23cf" 
        "\ufe0f" 
        "\u3030" 
        "]+", 
        flags=re.UNICODE
    )
    return emoji_pattern.sub("", text)

def remove_mentions(text):
    return re.sub(r'@\w+', '', text)

# def remove_hashtags(text):
#     return re.sub(r'#\w+', '', text)

def remove_newlines(text):
    return re.sub(r'\n+', ' ', text)

def remove_urls(text):
    return re.sub(r'http[s]?://\S+', '', text)

def remove_extra_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()

def normalize_arabic(text):
    text = re.sub(r'[إأآ]', 'ا', text)
    text = re.sub(r'[ىئ]', 'ي', text)
    return text

corrections = {
    "قوقل": "جوجل",
    "سامسنوق|سامسونق|سامسنوج|سامسونغ|ساموسنج|السامسونج": "سامسونج",
    "اندريود|اندوريد|اندرريد|الاندرويد|الاندريد": "اندرويد",
    "الجالكسي|الجلكسي|جالاكسي|الجـالكـسي": "جالكسي",
    "الايفون": "ايفون",
    "اونز": "اونر",
    "ويندز|الويندز": "ويندوز",
    "الايباد": "ايباد",
    "الساعه": "ساعة",
    "البلاگ": "البلاك",
    "الحوال|جوالي": "الجوال"
}

def correct_words(text):
    for wrong_pattern, right in corrections.items():
        text = re.sub(wrong_pattern, right, text)
    return text
    
def lower_english_letters(text):
    return text.lower()

def preprocess(text):
    text = remove_emojis(text)
    text = remove_mentions(text)
    #text = remove_hashtags(text)
    text = remove_newlines(text)
    text = remove_urls(text)
    text = remove_extra_spaces(text)
    text = normalize_arabic(text)
    text = correct_words(text)
    text = lower_english_letters(text)
    return text

In [28]:
# Text Preprocessing
df['text'] = df['text'].apply(lambda x: preprocess(x))
df.sample(5)

,text,comp,en1,en2
2844,تواصل معنا واحصل علي موقع الكتروني + تطبيق يعم...,0,None,None
328,الحمير الوحشيه ابطال السيريا بي اليوم يتوجون ب...,1,بكامرة كانون,كامرة الموبايل
22,ديل افضل من اسوس معي من ٤ سنًوات وشغال كويس ول...,1,ديل,اسوس
2228,بلاش المقارنة مع سوني، افضل من 360؟ فعلا القرد...,1,سوني,360
4433,هواوي تُعلن رسميا عن افضل هواتفها لهذا العام.....,0,None,None


In [29]:
# Entities Cleaning
print(df['en1'].apply(type).value_counts())
print(df['en2'].apply(type).value_counts())

en1
<class 'str'>         2326
<class 'NoneType'>    2272
<class 'int'>            1
Name: count, dtype: int64
en2
<class 'NoneType'>    2272
<class 'str'>         1812
<class 'float'>        512
<class 'int'>            3
Name: count, dtype: int64


In [30]:
df.loc[:, 'en1'] = df['en1'].apply(lambda x: preprocess(str(x)))
df.loc[:, 'en2'] = df['en2'].apply(lambda x: preprocess(str(x)))

In [31]:
print(df['en1'].apply(type).value_counts())
print(df['en2'].apply(type).value_counts())

en1
<class 'str'>    4599
Name: count, dtype: int64
en2
<class 'str'>    4599
Name: count, dtype: int64


In [32]:
df.sample(5)

,text,comp,en1,en2
4487,ثريد شرح موقع الماني متخصص بالاجهزة الالكتروني...,0,none,none
253,موقع تواصل تابع ل جوجل افضل من الفيسبوك بكثير ...,1,جوجل,الفيسبوك
1286,بالنسبة لتجربتي مع هواوي فيه تطبيقين واتساب و ...,1,هواوي,ايفون
1147,تمام انا عندي هواوي افضل من سامسونع بكثير سامس...,1,هواوي,سامسونج
4461,باكر راح نشهد انتهاء حقبة الشاشات المنحنية عند...,0,none,none


In [33]:
# Convert all Empty to 'nan'
def replace_to_nan(value):
    if value in ['NaN', 'none']:
        return 'nan'
    return value

df['en1'] = df['en1'].map(replace_to_nan)
df['en2'] = df['en2'].map(replace_to_nan)

In [34]:
# Get Entities not in Text
def split_values(val):
    if val == "nan":
        return []
    return [item.strip() for item in re.split(r'[ ,]+', val) if item.strip()]

df['check_condition'] = df.apply(
    lambda row: (
        ((any(val in row['text'] for val in split_values(row['en1']))) or row['en1'] == "nan") 
        and
        ((any(val in row['text'] for val in split_values(row['en2']))) or row['en2'] == "nan")
    ), axis=1)

filtered_df = df[df['check_condition']]
missing_rows = df[~df['check_condition']]
df.drop(columns=['check_condition'], inplace=True)

In [35]:
print(filtered_df.shape)
filtered_df.head()

(4585, 5)


,text,comp,en1,en2,check_condition
0,لابتوب اسوس افضل من لابتوبات ابل ؟ احس مستحيل ...,1,اسوس,ابل,True
1,اسوس افضل من وجهة نظري,1,اسوس,nan,True
2,كلهم نفس المواصفات مافي فرق اسوس افضل من hp في...,1,اسوس,hp,True
3,التخصصات الهندسية بالذات الحاسب والبرمجيات ماي...,1,اسوس,nan,True
4,اسوس افضل من ناحية التنوع والاسعار والعروض,1,اسوس,nan,True


In [36]:
print(missing_rows.shape)
missing_rows

(14, 5)


,text,comp,en1,en2,check_condition
227,بالنسبة لي جوجل افضل من 100 مع احترامي للجميع,1,افضل,,False
590,بالنسبة لي ابل افضل من ناحية برامج التصميم. بس...,1,ابل,سامسونج,False
659,اياً كان التشابه في النظامين ابل تصنعه بطريقة ...,1,ابل,,False
721,دايما حقين اندرويد عذرهم نظامiosخفيف واندرويد ...,1,معالجات ابل,,False
741,تبقي ابل افضل من اجهزة الاتاري والقيمز سامسونج,1,ابل,,False
755,بغض النظر عن السرعه و العلام.. نظام ال ios في ...,1,ios في ابل,اندرويد,False
871,افضل من سامسونج,1,ابل,سامسونج,False
1089,للان لو تتكلم عن شحن لاسلكي مافي شحن لاسلكي عن...,1,هواوي,ايفون,False
1168,اجمل شركه جوالات هواوي. افضل من,1,هواوي,apple,False
1474,مسؤول سابق في جوجل يؤكد: هواتف «ايفون» افضل من...,1,ايفون,جالكسي,False


# Save Files

In [37]:
# Total Data
df.to_csv("data.csv", index=False)

In [38]:
# com data
cleaned_com = df[df['comp'] == 1]
cleaned_com.shape

(2327, 4)

In [39]:
cleaned_com.to_csv("cleaned_com.csv", index=False)

In [40]:
# non data
cleaned_non = df[df['comp'] == 0]
cleaned_non.shape

(2272, 4)

In [41]:
cleaned_non.to_csv("cleaned_non.csv", index=False)